# DC motor tesztelése

Minden gyerek egyik legmeghatározóbb ismerkedése az elektromossággal az elemmel működő guruló játékautó. Ha zenélni és villogni is tud, akkor még érdekesebb. Ezt a játékot egy pici de egyszerű egyenáramú (DC) motor hajtja meg. Szinte mindannyiunk otthonában akad legalább egy belőle. 

Az első folytonosan működő DC motort Jedlik Ányos készítette el még 1827-ben. Érdekesség kép megjegyezzük, hogy 1855-ben épített vele egy model elektorom autót. Azóta a motorok kivitelezése sokat fejlődött, de a működési elvük nem változott. 

A motor forgatásához (típustól függően) elég egy 6 V körüli elem. Használat előtt érdemes elolvasni a motor használati útmutatóját, nehogy leégessük. Az elem egyik végét a motor egyik kimenetére, míg a másik végét a motor másik kimenetére kell kapcsolni és már forog is a motor. Ha felcseréljük a drótokat az elemen, akkor a motor a másik irányba forog. 

Ebben a projektben megismerkedünk egy olyan elektronikai kapcsolással, ami lehetővé teszi a motor biztonságos forgatását mindkét irányba, anélkül, hogy drótokat kellene áthelyezni, illetve megtanuljuk változtatni a motor forgási sebességét. Emellett elsajátítjuk azokat a parancsszavakat, amik lehetővé teszik, hogy vezéreljük a motort a Raspberry Pi-on keresztül. 

## Mit fogsz készíteni?

Egy 3-6V DC Motorból és egy L293D vezérlő (driver) chipből álló rendszert építünk, amit a Raspberry Pi-hoz kapcsolva vezérelni tudunk majd. **Megjegyzés: Mindig használjunk megfelelő drivert a motorral összekötve, hogy csökkentsük bármelyik eszköz károsodásának lehetőségét.**

## Mit tanulsz meg?

A DC motor és a hozzátartozó driver elkészítésével a következőket tanulod meg:

* Hogyan tudsz egy DC motort egy áramkörbe beiktatni a hozzá tartozó driverrel együtt.
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal

## A projekt részletekre bontása

* Elkészíteni az áramkört
* Inicializálni a Motor objektumot
* Terminálban tesztelni az objektumhoz tartozó parancsokat: forward, backward, reverse, speed, stop.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) DC Motor: **kijavitani a linkeket** [itt vásárolhatsz](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) L293D vezérlő: **kijavitani a linkeket** [itt vásárolhatsz](https://www.tme.eu/hu/katalog/tht-led-diodak-5mm_112898/?s_order=desc&search=led&s_field=1000011)

## A kapcsolási rajz

<img src="prog01_circuit.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t.

1) A nyomógomb lábait illesszük a breadbord mélyedéseibe. Ügyeljünk arra, hogy a nyomógomb egy (vízszintes) sorba kerülő lábai a gomb lenyomása nélkül is kapcsolatban legyenek egymással, míg a (függőlegesen) egy oszlopba kerülő lábai csak a lenyomás következtében legyenek összekötve. Mindezt egy műszerrel leellenőrízhetjük.

2) Helyezzük a LED lábait két különboző sorba. A katódját (negatív láb) kössük a földeléshez míg az anódjának (pozitív láb) sorába kössük be az ellenállásunk egyik lábát, míg a másikat egy üresen álló sorba tegyük. A LED anódjának és katódjának megállapítására használhatunk egy multimétert. Csatlakoztasd a fekete mérőzsinórt a COM (föld) és a piros mérőzsinórt a VΩmA jelzésű hüvelybe. Forgasd el a méréshatárváltó kapcsolót a folytonosság mérés állapotba. Érintsd a két zsínórt a két lábhoz és ha a LED halványan pislákol, akkor az a láb amelyikhez a fekete zsinórt érintetted a katód, a másik az anód. Ha nem pislákol akkor cseréld meg a zsinórok és a lábak érintkezését, hogy világítson.

3) Nevezzük ki a breadboard oldalsó oszlopainak egyikét a közös földelésnek. Ide kössük be a nyomógomb egyik lábáz illetve a LED anódját. Ugyanebbe az oszlopba kössük be a Raspberry Pi egyik **GND** jelölésű tüskéjét is egy jumper segítségével.

4) A nyomógomb eddig szabadon lévő lábát egy jumperrel kössük össze a **2**-es GPIO tüskével.

5) Az ellenállás szabadon levő lábát egy jumperrel kössük össze a **14**-es GPIO tüskével.

## A kód
Nyissunk meg egy új python fájlt és mentsük el pl. ```dcmotor_test.py``` név alatt.

### A DC motor kipróbálása

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, a motor mozgatására.

Először betöltjük a ```gpiozero``` modulból a ```Motor``` objektumot ami lehetővé teszik, hogy a Raspi kommunikáljon a motorral a hozzá tartozó vezérlővel a **GPIO** tüskéken keresztül. Emellett beimportáljuk a ```sleep``` függvényt is, hogy szünetet tarthassunk a python kód lefutásában. 

N.B. Az itt közölt parancsokat akár a terminálba is gépelhetjük, miután elindítottuk az ```ipython``` vagy ```python``` környezetet.

```dcmotor_test.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

Az objektum beolvasása után inicializálunk egy ```Motor``` objektumot, amit ```motor```-nak nevezünk el. Egyben megmondjuk az objektumnak, hogy a fizikai motor előre forgatásért felelő drótját a *23*-as számú **GPIO** tüskére csatlakoztattuk, míg a hátrafelé forgatásért felelő drótot a *24*-es számú **GPIO** tüskére. Emellett a driveren a kimeneteket aktiváló drótot a *25*-ös számú **GPIO** tüskére csatlakoztattuk. 

```dcmotor_test.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)

Ezután készen állunk arra, hogy leteszteljük a motor mozgatását és az objektumhoz tartozó egyéb metódusok működését. Elsőnek a motort előre forgatjuk ```motor.forward()```, 5 másodpercen keresztül. Majd lecsökkentjük a forgás sebességét, ```motor.forward(speed=0.8)```. A ```speed``` értéke 0 és 1 között változhat, de ha túl kicsire vesszük a sebességet, nem biztos, hogy a motor rendelkezni fog elég nyomatékkal, hogy forogjon, ami ha tartós ideig áll fenn, akkor leéghet a motor. Ezután leellenőrizzük, hogy a motor valóban forog-e még a ```motor.is_active``` metódussal. A forgási sebességet meghatározó értéket a ```motor.value``` metódussal kaphatjuk vissza, amit ki is nyomtatunk a képernyőre. Majd hadjuk forogni a motort 5 másodpercig, és megállítjuk a motort, ```motor.stop()```.

```dcmotor_test.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)

motor.forward()
sleep(5)

motor.forward(speed=0.8)
print(motor.is_active)
print(motor.value)
sleep(5)
motor.stop()

A motorhoz tartozó további metódusok gyakorlássaképp próbáljuk ki hátrafele forgatni a motor kerekét a ```motor.backward()``` paranccsal, ahol szintén megadhatjuk a forgási sebességet a ```speed``` argumentum segítségével. Ezután kinyomtathatjuk a sebesség értékét, hogy lássuk helyesen lett beállítva és hadjuk a motort forogni 5 másodpercen keresztül. A ```motor.reverse()``` paranccsal a forgási irányt tudjuk változtatni, miközben a beállított sebesség nem változik. Hagyjuk ismét a motort forogni 5 másodpercig. Ezután megállítjuk a motort a ```motor.stop()``` paranccsal és a ```motor.is_active``` paranccsal leellenőrizzük, hogy valóban leállt-t. Ha az értéke ```False```, akkor sikeresen megállítottuk a motort. 

```dcmotor_test.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)

motor.forward()
sleep(5)

motor.forward(speed=0.8)
print(motor.is_active)
print(motor.value)
sleep(5)
motor.stop()

motor.backward(speed=0.9)
print(motor.value)
sleep(5)
motor.reverse()
sleep(5)
motor.stop()
print(motor.is_active)

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```dcmotor_test.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```dcmotor_test.py```-t elmentettük. Ott begépelve a ```python dcmotor_test.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a gomb lenyomására a LED dióda világít, majd újbóli lenyomására a motor elkezd forogni, majd megváltozik a sebessége, majd megáll. Utána ismét forogni kezd, csak az ellenkező irányba, majd ismét irányt vált és végül megáll.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Teszteljük le, mekkora sebességgel képes forogni a motor.
* Változtassuk meg az időt, amíg a motor az egyik, majd a másik irányba forog.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) LED objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#motor](https://gpiozero.readthedocs.io/en/stable/api_output.html#motor)

3) L293D driver leírása - [https://www.ti.com/lit/ds/symlink/l293.pdf](https://www.ti.com/lit/ds/symlink/l293.pdf)